# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [9]:
# Dependencies and Setup
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

ModuleNotFoundError: No module named 'requests'

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame


In [4]:

data_path="output_data/cities.csv"
cities=pd.read_csv(data_path)
cities

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [ ]:

gmaps.configure(api_key=g_key)

In [ ]:
gmaps.configure(api_key=g_key)


locations = cities[["Latitude", "Longitude"]]
humidity = cities["Humidity (%)"]
fig = gmaps.figure(zoom_level=1.6, center= (20, 0))

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1,
                                )

fig.add_layer(heat_layer)
fig

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [2]:
ideal_city = cities.loc[(cities["Temperature"]<80) & (cities["Temperature"]>75) & (cities["Wind Speed (mph)"]<10)& (cities["Cloudiness (%)"]==0) & (cities["Wind Speed (mph)"]<9)].dropna()
ideal_city

NameError: name 'cities' is not defined

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [1]:
#creating variable hotel_df
hotel_df = ideal_city[['City','Country','Latitude','Longitude']].copy()


NameError: name 'ideal_city' is not defined

In [ ]:
# geocoordinates
target_radius = 5000
target_type = "lodging"

# set up a parameters dictionary
params = {
    "radius": target_radius,
    "type": target_type,
    "key": g_key
}

for index, row in hotel_df.iterrows():
    lat = row['Lat']
    lng =row['Lng']
    params['location'] = f"{lat},{lng}"
    # base url
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # run a request using our params dictionary
    response = requests.get(base_url, params=params).json()
    
    try:
        hotel_df.loc[index, "Hotel Name"] = response['results'][0]['name']
    except (KeyError, IndexError):
        print(f"no hotel in {index}")
print(hotel_df)
hotel_df.to_csv("output_data/hotel_data")

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations)

# Display figure
fig.add_layer(markers)
fig